# Assignment 6

### Part A

In [11]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [12]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [13]:
def exp_return(data, P):
    result = (data["Probability"] * (data['Price in 1 Year']/P - 1)).sum()
    return result

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [14]:
def volatility(data, P):
    expected_return = (data["Price in 1 Year"] * data["Probability"]).sum()
    var = ((data["Price in 1 Year"] - expected_return) ** 2 * data["Probability"]).sum()
    volatility = (var ** (1/2)) / P
    return volatility

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [15]:
def interval(data, P):
   std_error = (volatility(data, P)) / (len(data) ** 0.5)
   lower_bound = exp_return(data, P) - (1.96 * std_error)
   upper_bound = exp_return(data, P) + (1.96 * std_error)
   return [lower_bound, upper_bound]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [16]:
def exp_return_capm(beta, market_return, rf):
    expected_return = rf + beta * (market_return - rf)
    return expected_return

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [17]:
def clean_data(path):
    df = pd.read_csv(path)
    # remove non-number RET
    # Convert all columns to numeric values (coerce non-numeric to NaN)
    df["RET"] = df["RET"].apply(pd.to_numeric, errors='coerce')
    df = df[pd.notnull(df['RET'])]
    # remove securities < 500
    permno_counts = df['PERMNO'].value_counts()
    valid_permnos = permno_counts.index[permno_counts >= 500]
    df = df[df['PERMNO'].isin(valid_permnos)]
    return df

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [18]:
def avg_return(data):
    avg_returns = data.groupby('PERMNO')['RET'].mean().reset_index()
    avg_returns.columns = ['PERMNO', 'Avg_Return']
    
    output = pd.DataFrame(data = {
        "CUSIP" : avg_returns['PERMNO'],
        "Average Return" : avg_returns['Avg_Return']
    })
    
    return output
#data = clean_data(r"C:\Users\Amy\Downloads\monthlycrsp_mod6.csv")
#avg_return(data)

In [19]:
def security_volatilities(data):
    volatilities = data.groupby('PERMNO')['RET'].std().reset_index()
    volatilities.columns = ['PERMNO', 'Volatility']
    output = pd.DataFrame(data = {
        "CUSIP" : volatilities['PERMNO'],
        "Volatility" : volatilities['Volatility']
    })
    return output
#data = clean_data(r"C:\Users\Amy\Downloads\monthlycrsp_mod6.csv")
#security_volatilities(data)

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [20]:
def correlation(ret_data, vol_data):
    merged_data = pd.merge(ret_data, vol_data, on='CUSIP')
    corr = merged_data['Average Return'].corr(merged_data['Volatility'])
    return corr
#data = clean_data(r"C:\Users\Amy\Downloads\monthlycrsp_mod6.csv")
#ret_data = avg_return(data)
#vol_data = security_volatilities(data)
#correlation(ret_data, vol_data)